# Seed plans demo!

First, we'll do a simple example using `recursive_seed_part`. 

Next, we'll dive into how the parameters change the function's behavior, and some heuristics for when to try different settings. 

Finally, we'll explore what to do if `recursive_seed_part` doesn't quickly return a partition of desired population balance by viewing seed plan generation as a multi-step process.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from gerrychain.tree import recursive_tree_part, recursive_seed_part, bipartition_tree, bipartition_tree_random #,recursive_seed_part
import numpy as np

## Part I. A first example

###  Load the graph we want to use

First, git clone the CO shapefile from https://github.com/mggg-states/CO-shapefiles

In [ ]:
# change this path to locate your CO shapefiles
graph = Graph.from_file("./../states/CO-shapefiles/co_precincts.shp")

### Find the total population of the region

Here, we use any existing partition (with any number of districts) to compute `total_population`, but one can loop over units if there is no partition.

In [ ]:
pop_col = "TOTPOP"
existing_partition = "CD116FP"
place_name = "CO"

my_updaters = {"population": updaters.Tally(pop_col, alias="population")}
initial_partition = GeographicPartition(graph, assignment=existing_partition, updaters = my_updaters)
total_population = sum(initial_partition["population"].values())

### Generate a seed plan!

Here, we are generating a plan with `num_dists = 11` districts within `epsilon = 2%` population balance.

In [ ]:
# set the desired number of districts and population deviation
num_dists = 11
epsilon = .02

In [ ]:
# generate the plan! 
pop_target = total_population / num_dists
result = recursive_seed_part(graph, 
                             range(num_dists), 
                             pop_target, 
                             pop_col, 
                             epsilon, 
                             method=bipartition_tree, 
                             n = None, 
                             ceil = None)

In [ ]:
# plot the plan
partition = GeographicPartition(graph, result, updaters=my_updaters)
partition.plot(cmap="tab20")

## Part II. How to generate your own seed plan using `recursive_seed_part`

To generate a seed plan using `recursive_seed_part`, there are two parameters to keep in mind: `n` and `ceil`.

The function `recursive_seed_part` works by recursively splitting a region into smaller chunks, and then splitting those chunks into smaller chunks, until the ideal district size is reached. These two parameters allow you to control the number of chunks a region will be split into at each level.

See a visual example of these three cases here: https://github.com/blumt/seed_plans/blob/main/seed_plan_examples.pdf

### Case 1. If `n` is `None` and `ceil` is `None`

If both `n` and `ceil` are set to `None` (this is the default state), then let `k_1*k_2*...*k_m` be the prime factorization of the desired number of districts, where `k_i` are in decreasing order. The region is first split into `k_1` chunks, then each chunk is split into `k_2` chunks, and so on.

In general, these parameters work well when the number of districts has many small prime factors. This may not work as well when the number of districts is a large prime, or is a product of a few large primes.

### Case 2. If `n` is a positive integer greater than 1

In this case, `ceil` is irrelevant. In this case, the region is recursively split into districts as follows: 
* if the number of districts a region should be split into is divisible by `n`, the region is divided into `n` balanced chunks; 
* otherwise, there is a remainder `r` when the number of districts is divided by `n`. Bite off `r` districts from the region, and then split the remaining region into `n` chunks.
Continue recursively splitting the chunks into districts as above.

This setting works best when `n` is a small prime, such as 2,3, or 5. This setting works well in a general case.

### Case 3. If `n` is `None` and `ceil` is a positive integer greater than 2

In this case, `ceil` is an upper bound on the number of chunks a region can be split into at any given step. The region is recursively split into districts as follows. Let `k` be the largest prime factor of the number of districts a region is to be split into which is at most `ceil`. 
* If `k` exists, then split the region into `k` balanced chunks
* If `k` does not exist (i.e. the number of districts has no prime factor less than `ceil`), then bite off one district from the region; the remaining region is now a chunk.
Continue recursively splitting chunks into districts as above.

This setting works well in a general case. If the desired number of districts has many factors that are small primes, but one or two large prime factors, setting `ceil` to be 5 or 7 can be effective. As `ceil` is large, this is similar to case 1 above, and as ceil is small this is similar to case 2.

### Example(s)

Play around with the number of districts, population balance, `n`, and `ceil` in the example below! 
* What settings work best when the number of districts is set to a large prime, like 89 or 97?
* What works best when the number of districts has a nice prime factorization, like 75?
* What if the number of districts has one large prime factor, like 38?

In [ ]:
num_dists = 75
epsilon = .02
pop_target = total_population / num_dists
result = recursive_seed_part(graph, 
                             range(num_dists), 
                             pop_target, 
                             pop_col, 
                             epsilon, 
                             method=bipartition_tree, 
                             n = None, #    <-- try changing this!
                             ceil = None) # <-- try changing this!
# plot the plan
partition = GeographicPartition(graph, result, updaters=my_updaters)
partition.plot(cmap="flag")

## Part III. But `recursive_seed_part` isn't working for my seed plan!

You might be looking for a seed plan with 293 districts or balanced within .5% population deviation. In cases with a large number of districts and/or a very small goal population balance, `recursive_seed_part` might be slow or ineffective at finding a solution to your seed plan woes. Luckily, there are ways around this.

To get around this, here's a two-step process: first, generate a seed plan with the desired number of districts, but with a too-high population deviation. Next, run a chain on the initial partition until you get a plan that is within the desired population balance. This method, and the code for it, is due to Amy Becker.

### Step 1. Generate an initial plan
The goal of this step is to generate an initial plan with the desired number of districts, with little to no regard for population deviation. You have a few options. 
* You can use `recursive_seed_part` or `recursive_tree_part` to generate an initial plan by setting `epsilon` (the population deviation) to be higher than your desired balance. 
* You can completely disregard population balance by randomly assigning `num_dists - 1` precincts to be entire districts, and let the final district be the remaining region. 
* Or any other way you can think of! (e.g. draw it in Districtr, use some other method, etc)

### Step 2. Run a chain to lower the population deviation to within the desired range
Using the initial partition from step 1, run a chain to lower the population deviation by merging two adjacent districts and then dividing the merged region into two new balanced districts. 

In general, this step will be quicker if the initial plan from step 1 already has a smaller population deviation.

### Examples: 

In [ ]:
# import code to generate random initial plans, and to run targeted chains
# these methods are due to Amy Becker
from seed_plans_iterative_bipartition import gen_initial_partition, pop_shuffle, population_deviation

#### Example using `recursive_seed_part` to generate the initial partition

The goal of this example is to compute a 65-district seed plan for Colorado within 2% population deviation.

In [ ]:
# Step 1: Generate an initial plan using recursive_seed_part balanced within 10%
num_dists = 65
epsilon = .1
pop_target = total_population / num_dists
result = recursive_seed_part(graph, 
                             range(num_dists), 
                             pop_target, 
                             pop_col, 
                             epsilon, 
                             method=bipartition_tree, 
                             n = None, 
                             ceil = None)
# plot the plan
partition = GeographicPartition(graph, result, updaters=my_updaters)
partition.plot(cmap="flag")

In [ ]:
# Step 2: Run a targeted chain to lower the population deviation
pop_dev = .02
for i in range(9000):
    partition = pop_shuffle(partition, pop_col, pop_dev/2, weights = True)
    if population_deviation(partition) < pop_dev: #
        print(i, population_deviation(partition))
        break
    if i % 100 == 0:
        print(i, population_deviation(partition))

# plot the plan
partition.plot(cmap="flag")

#### Example by randomly assigning nodes to generate the initial plan

The goal again is to generate a 65-district plan for CO within 2% population deviation.

In [ ]:
# Step 1: Generate the initial plan
num_dists = 65
partition = gen_initial_partition(graph, num_dists, my_updaters)
# plot the initial plan
partition.plot(cmap="prism")

In [ ]:
# Step 2: Run a targeted chain to lower the population deviation
pop_dev = 0.02
#first get down to be reasonably balanced
for i in range(1000):
    partition = pop_shuffle(partition, pop_col, 0.10, weights = True)
    if population_deviation(partition) < pop_dev:
        print(i, population_deviation(partition))
        break
    if i % 100 == 0:
        print(i, population_deviation(partition))
        
#then refine balance
for i in range(10000):
    partition = pop_shuffle(partition, pop_col, pop_dev/2, weights = True)
    if population_deviation(partition) < pop_dev:
        print(i, population_deviation(partition))
        break
    if i % 100 == 0:
        print(i, population_deviation(partition))

# plot the plan
partition.plot(cmap="prism")